# 5. Preprocessing a surface


Pit filling and swamp filling ... 

Take the previous mesh with random noise 

`Quagmire` allows the user to specify the number of downhill pathways to model landscape evolution. This is set using:

```python
mesh.downhill_neighbours = 1
mesh.update_height(height)
```

where an integer specifies the number of downhill neighbour nodes (receipients) that will receive a packet of information from a donor node. The `QuagMesh` object can also be initialised with:

```python
mesh = QuagMesh(DM, downhill_neighbours=1)
```

to specify the number of downhill neighbours (default is 2).

In this notebook we use a landscape function with many outflow points to examine the effect of varying the number of recipient nodes on catchment area, stream lengths, and outflow fluxes.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from quagmire import tools as meshtools
from quagmire import function as fn
%matplotlib inline

In [2]:
from quagmire import QuagMesh 
from quagmire import QuagMesh # all routines we need are within this class
from quagmire import QuagMesh


minX, maxX = -5.0, 5.0
minY, maxY = -5.0, 5.0,

spacingX = 0.05
spacingY = 0.05

x, y, simplices = meshtools.elliptical_mesh(minX, maxX, minY, maxY, spacingX, spacingY, 1.)

DM = meshtools.create_DMPlex(x, y, simplices, refinement_levels=2)

mesh = QuagMesh(DM)

x = mesh.coords[:,0]
y = mesh.coords[:,1]

print( "\nNumber of points in the triangulation: {}".format(mesh.npoints))

Underlying Mesh type: TriMesh
0 - Delaunay triangulation 0.46267416599999933s
0 - Calculate node weights and area 0.043066750000001264s
0 - Find boundaries 0.02290166599999921s
0 - cKDTree 0.02247120799999891s
0 - Construct neighbour cloud arrays 0.6619863330000015s, (0.44069049999999876s + 0.2212610419999983s)
0 - Construct rbf weights 0.05931295799999958s

Number of points in the triangulation: 156925


In [3]:
# topography

radius  = np.sqrt((x**2 + y**2))
theta   = np.arctan2(y,x)+0.1

height  = np.exp(-0.025*(x**2 + y**2)**2) + 0.25 * (0.2*radius)**4  * np.cos(5.0*theta)**2 ## Less so
height  += 0.5 * (1.0-0.2*radius)
height  -= height.min()

## Add some pits 

hrand0 = np.where(np.random.random(height.shape)>0.995, -0.1, 0.0)


## Add smoothed random noise to make some "lakes" 

rbf_smoother = mesh.build_rbf_smoother(0.05, iterations=3)
h0 = mesh.add_variable(name="h0")
h0.data = np.where(np.random.random(height.shape)>0.995, -0.33, 0.0)

hrand1 = 25.0 * rbf_smoother.smooth_fn(h0, iterations=25).evaluate(mesh)


# randpts1 = np.where(np.random.random(height.shape)>0.995, -1.0, 0.0)
# hrand1   = 10.0 * rbf_smoother.smooth(randpts1, iterations=10)

heightn = np.maximum(0.0, height + hrand0 + hrand1)

In [4]:
with mesh.deform_topography():
    
    mesh.downhill_neighbours = 2
    mesh.topography.data = heightn

# let's use a rainfall proportional to height (any choice is ok)

rainfall_fn = mesh.topography**2

low_points = mesh.identify_low_points()
low_point_coords = mesh.coords[low_points] 
print(low_points.shape)

0 - Build downhill matrices 3.843434333000001s
0 - Build upstream areas 1.5636943749999972s
(886,)


In [5]:
h0.data = mesh.topography.data

mesh

In [6]:
import k3d

lowsxyz = np.column_stack([mesh.tri.points[low_points], heightn[low_points]])
xyz  = np.column_stack([mesh.tri.points, height])
xyz2 = np.column_stack([mesh.tri.points, heightn])

plot = k3d.plot(camera_auto_fit=False, grid_visible=False, lighting=0.75)

indices = mesh.tri.simplices.astype(np.uint32)

surface = k3d.mesh(xyz2, indices, wireframe=False, flat_shading=True, side="double",
                    color_map = k3d.colormaps.matplotlib_color_maps.Cividis,
                    attribute=mesh.topography.data,
                    color_range = [0.0,1.5]
                   )

lows    = k3d.points(lowsxyz, point_size=0.05, color=0xFF0000)

plot += surface + lows

plot.display()



/Users/lmoresi/mambaforge/envs/underworld/lib/python3.9/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()

In [7]:
rainfall_fn = mesh.topography**2
flowrate_fn = mesh.upstream_integral_fn(rainfall_fn)
stream_power_fn = flowrate_fn ** 2.0 * mesh.slope ** 2.0 * mesh.mask

cumulative_rain_n1 = mesh.upstream_integral_fn(rainfall_fn).evaluate(mesh)

In [8]:

import k3d

lowsxyz = np.column_stack([mesh.tri.points[low_points], heightn[low_points]])
xyz  = np.column_stack([mesh.tri.points, height])
xyz2 = np.column_stack([mesh.tri.points, heightn])
indices = mesh.tri.simplices.astype(np.uint32)



plot = k3d.plot(camera_auto_fit=False, grid_visible=False, lighting=0.75)


surface = k3d.mesh(xyz2, indices, wireframe=False, flat_shading=True, side="double",
                    color_map = k3d.colormaps.matplotlib_color_maps.Blues,
                    attribute=stream_power_fn.evaluate(mesh),
                    color_range = [0.0,0.01]
                   )

lows    = k3d.points(lowsxyz, point_size=0.05, color=0xFF0000)

plot += surface + lows

plot.display()



Output()

In [98]:
## Pit filling algorithm in quagmire

mesh1p = QuagMesh(DM)

rainfall_fn_1p = mesh1p.topography**2
cumulative_rain_n1p = mesh1p.upstream_integral_fn(rainfall_fn_1p)
stream_power_n1p    = cumulative_rain_n1p ** 2.0 * mesh1p.slope ** 2.0 * mesh1p.mask



with mesh1p.deform_topography():
    mesh1p.topography.data = mesh.topography.data

Underlying Mesh type: TriMesh
0 - Delaunay triangulation 0.3040237499999421s
0 - Calculate node weights and area 0.0241118330000063s
0 - Find boundaries 0.01836920799996733s
0 - cKDTree 0.019008834000032948s
0 - Construct neighbour cloud arrays 0.6636431250001351s, (0.43505416700008936s + 0.2285542079998777s)
0 - Construct rbf weights 0.06518099999993865s
0 - Build downhill matrices 2.430833333999999s
0 - Build upstream areas 1.5887439580001228s


In [99]:
mesh1p.low_points_local_patch_fill(its=2, smoothing_steps=1, fraction=0.25)

Low point local patch fill
0 - Build downhill matrices 2.4737774160000754s
0 - Build downhill matrices 2.4994586660000095s
Low point local patch fill  7.116479208999863  seconds
0 - Build upstream areas 1.5854269160001877s


In [100]:
mesh1p.identify_low_points().shape

(193,)

In [121]:
import k3d

low_points2 = mesh1p.low_points
height2 = mesh1p.topography.data

lowsxyz = np.column_stack([mesh1p.tri.points[low_points], heightn[low_points]])
lows2xyz = np.column_stack([mesh1p.tri.points[low_points2], height2[low_points2]])

xyz  = np.column_stack([mesh.tri.points, mesh1p.topography.data])
xyz2 = np.column_stack([mesh.tri.points, heightn])
indices = mesh.tri.simplices.astype(np.uint32)

plot = k3d.plot(camera_auto_fit=False, grid_visible=False, lighting=0.75)


surface = k3d.mesh(xyz, indices, wireframe=False, flat_shading=True, side="double",
                    color_map = k3d.colormaps.matplotlib_color_maps.Blues,
                    attribute=cumulative_rain_n1p.evaluate(mesh),
                    color_range = [0.0,1.0]
                   )

lows    = k3d.points(lowsxyz, point_size=0.05, color=0xFF0000)
lows2    = k3d.points(lows2xyz, point_size=0.05, color=0x0000FF)


plot += surface + lows + lows2 

plot.display()


Output()

In [105]:
## Quagmire also has a swamp filling algorithm
## NOTE this is much more efficient if it follows the pit filling

mesh1s = QuagMesh(DM)
rainfall_fn_1s = mesh1s.topography**2

with mesh1s.deform_topography():
    mesh1s.topography.data = mesh1p.topography.data
    
print("There are still {} low points".format(mesh1s.identify_low_points().shape))

Underlying Mesh type: TriMesh
0 - Delaunay triangulation 0.3330004590000044s
0 - Calculate node weights and area 0.0643369580000126s
0 - Find boundaries 0.018808749999834617s
0 - cKDTree 0.0196499579999454s
0 - Construct neighbour cloud arrays 0.6992000420000295s, (0.44694904200014207s + 0.25221525000006295s)
0 - Construct rbf weights 0.058850083999914204s
0 - Build downhill matrices 2.4509212079999543s
0 - Build upstream areas 1.571919041000001s
(193,)


In [106]:
for i in range(0,50):
    mesh1s.low_points_swamp_fill(ref_height=-0.01)
    
    # In parallel, we can't break if ANY processor has work to do (barrier / sync issue)
    low_points2 = mesh1s.identify_global_low_points()
    
    print("{} : {}".format(i,low_points2[0]))
    if low_points2[0] == 0:
        break

120  iterations, time =  1.2702137920000496
Build low point catchments -  1.2706606249998913  seconds
0  Sort spills -  0.010811040999897159
0  Gather spill data -  0.0019107919999896694
0  Sort all spills -  0.00026254099998368474
0 - Build downhill matrices 2.4735215000000608s
Low point swamp fill  3.997965583999985  seconds
0 - Build upstream areas 1.8586511250000513s
0 : 96
178  iterations, time =  1.3629859999998644
Build low point catchments -  1.3634352080000554  seconds
0  Sort spills -  0.005808292000210713
0  Gather spill data -  7.35829999030102e-05
0  Sort all spills -  9.187499995277904e-05
0 - Build downhill matrices 2.4238212090001525s
Low point swamp fill  3.935164916999838  seconds
0 - Build upstream areas 1.8560250830000768s
1 : 70
263  iterations, time =  1.4846040420000008
Build low point catchments -  1.485055124999917  seconds
0  Sort spills -  0.004419750000124623
0  Gather spill data -  6.962499992368976e-05
0  Sort all spills -  8.179099995686556e-05
0 - Build 

In [109]:
cumulative_rain_n1s = mesh1s.upstream_integral_fn(rainfall_fn_1s)
stream_power_n1s    = cumulative_rain_n1s ** 2.0 * mesh1s.slope ** 2.0 * mesh1s.mask

In [122]:
import k3d

low_points2 = mesh1p.low_points
height2 = mesh1p.topography.data

lowsxyz = np.column_stack([mesh1p.tri.points[low_points], heightn[low_points]])
lows2xyz = np.column_stack([mesh1p.tri.points[low_points2], height2[low_points2]])

xyz  = np.column_stack([mesh1s.tri.points, mesh1s.topography.data])
xyz2 = np.column_stack([mesh1p.tri.points, mesh1p.topography.data])
indices = mesh.tri.simplices.astype(np.uint32)

plot = k3d.plot(camera_auto_fit=False, grid_visible=False, lighting=0.75)


surface = k3d.mesh(xyz2, indices, wireframe=False, flat_shading=True, side="double",
                    color_map = k3d.colormaps.matplotlib_color_maps.Blues,
                    attribute=cumulative_rain_n1s.evaluate(mesh),
                    color_range = [0.0,1.0]
                   )

lakes_surface = k3d.mesh(xyz-(0.0,0.0,0.005), indices, wireframe=False, flat_shading=True, side="double",
                    color=0x0010C0, opacity=0.5
                   )



lows    = k3d.points(lowsxyz, point_size=0.05, color=0xFF0000)
lows2    = k3d.points(lows2xyz, point_size=0.05, color=0x0000FF)


plot += surface + lows + lows2 + lakes_surface

plot.display()

Output()